Log in to LinkedIn?



In [ ]:
var notRobot = () => {
    return client.click('')
}

var enterLinkedIn = () => {
    console.log('LinkedIn: Sign in required');
    var credentials = getCredentials('linkedin.com');
    return client.isExisting('a[href*="uas/login"]')
    .then(is => is ? client.click('a[href*="uas/login"]') : Promise.resolve([]))
    .then(() => client.click('input[name*="session_key"]'))
    .keys(credentials.username)
    .pause(1000)
    .then(() => console.log('LinkedIn: Require password'))
    .click('input[name*="session_password"]')
    .keys(credentials.password)
    .submitForm('.login-form, [type="submit"]')
    .pause(2000)
    .isExisting('.cp-challenge-form')
    .then(is => {
        if(is) {
            throw new Error('captcha');
        }
    });
}

var loginLinkedIn = () => {
    return client
    .getUrl().then(url => {
        var loggedIn = url.indexOf('linkedin') > -1 && url.indexOf('login') == -1 && url !== 'https://www.linkedin.com'
        return loggedIn
            ? client
                .pause(1000)
                .isExisting('iframe.authentication-iframe').then(is => is 
                    ? client.element('iframe.authentication-iframe')
                        .then(el => client.frame(el.value))
                        .then(() => enterLinkedIn())
                        .frame()
                    : Promise.resolve([]) )
            : client.url('https://www.linkedin.com/')
                .pause(1000)
                .isExisting('*=Forgot password?').then(is => is 
                    ? enterLinkedIn()
                    : Promise.resolve([]));
    })
};
module.exports = loginLinkedIn;
loginLinkedIn;


Scrape my own LinkedIn profile?



In [ ]:
var visitMyProfile = () => {
    // TODO: add check for needing to go to LinkedIn
    // TODO: add check for needing to login
    return client.click('#nav-settings__dropdown-trigger')
        .pause(500)
        .click('.nav-settings__view-profile-link')
        .pause(2000);
}

var readLinkedInProfileInfo = () => {
    var name, title, url, phone, email;
    return client
    .getUrl().then(url => url.indexOf('/in/') == -1
        ? visitMyProfile()
        : Promise.resolve([]))
    .isExisting('.contact-see-more-less')
    .then(is => is ? client.click('.contact-see-more-less') : client)
    .then(() => Promise.all([
        client.getText('.pv-top-card-section__name'),
        client.getText('.pv-top-card-section__headline'),
        client.getText('.ci-vanity-url .pv-contact-info__contact-item').catch(e => client.getUrl()),
        // is existing?
        client.getText('.ci-phone .pv-contact-info__contact-item').catch(e => null),
        client.getText('.ci-email .pv-contact-info__contact-item').catch(e => null)
    ]))
    .then(r => ({
        name: r[0],
        title: r[1],
        url: r[2],
        phone: r[3],
        email: r[4],
    }));
};
module.exports = readLinkedInProfileInfo;
readLinkedInProfileInfo;


List all LinkedIn threads?



In [ ]:
var scrollLinkedInThreads = (threads) => {
    // TODO: add check for needing to go to LinkedIn
    // TODO: add check for needing to login
    // scroll to bottom of messages
    return client
        .execute(() => {
            document.getElementsByClassName(
                'msg-conversations-container__conversations-list')[0]
                .scrollTop += 10000;
        })
        .pause(2000)
        .then(() => getLinkedInThreads(threads));
}

var getLinkedInThreads = (threads) => {
    return client    
    .execute(() => {
        var iterator = document.evaluate(
            '//*[@data-control-name="view_message"]/@href',
            document, null,
            XPathResult.UNORDERED_NODE_ITERATOR_TYPE, null);
        var co = []; var m;
        while ((m = iterator.iterateNext()) && co.push(m.value));
        return co;
    })
    .then(els => {
        var newThreads = els.value.filter(e => threads.indexOf(e) === -1);
        threads = threads.concat(newThreads);
        return newThreads.length > 0
            ? scrollLinkedInThreads(threads)
            : Promise.resolve(threads);
    });
}

var listLinkedInThreads = (threads) => {
    return client.getUrl().then(url => url.indexOf('/messaging') == -1
            ? client.click('a[href*="/messaging/"]')
            : Promise.resolve([]))
        .pause(2000)
        .then(() => getLinkedInThreads(threads || []))
};
module.exports = listLinkedInThreads;
listLinkedInThreads;


Read all messages in LinkedIn thread?



In [ ]:
var importer = require('../Core');
var moment = require('moment');
var chrono = require('chrono-node');

var getThreadParticipants = (thread) => {
    var profiles = [], alreadyAt = false;
    var readLinkedInProfileInfo;
    // check for thread url and go there
    return client.then(() => importer.interpretAll(['scrape LinkedIn profile']))
    .then(r => r[0].runFresherInNewContext({client}))
    .then(r => readLinkedInProfileInfo = r)
    .getUrl()
    .then(url => {
        return url.indexOf(thread + 'topcard/') == -1
            ? client.url('https://www.linkedin.com' + thread + 'topcard/')
            : Promise.resolve([]);
    })
    .pause(3000)
    .elements('.view-profile')
    .then(els => Promise.all(els.value.map(a => {
        return client.elementIdAttribute(a.ELEMENT, 'href').then(h => h.value);
    })))
    .then(links => {
        return importer.runAllPromises(links.map(link => (resolve => {
            var results = [];
            return client
            .click('a[href*="' + link.replace('https://www.linkedin.com', '') + '"]')
            .pause(3000)
            .then(() => readLinkedInProfileInfo(link))
            .then(r => results = r)
            .back()
            .pause(1000)
            .then(r => resolve(results))
            .catch(e => console.log(e))
        })));
    });
};

var scrollLinkedInMessages = (messages) => {
    // TODO: add check for needing to go to LinkedIn
    // TODO: add check for needing to login
    // scroll to bottom of messages
    return client
        .execute(() => {
            document.getElementsByClassName('msg-s-message-list')[0]
                .scrollTop -= 10000;
        })
        .pause(2000)
        .then(() => readLinkedInMessages(messages));
}

var readLinkedInMessages = (messages) => {
    // TODO: check for thread id in url?
    return client
    .execute(() => {
        var groups = document.getElementsByClassName(
            'msg-s-message-list__date-group');
        var result = [];
        for(var i = 0; i < groups.length; i++) {
            var times = groups[i].getElementsByTagName('time');
            var messages = groups[i].getElementsByClassName('msg-s-message-group');
            for(var j = 0; j < messages.length; j++) {
                var profile = messages[j].getElementsByTagName('img')[0];
                var sibling = messages[j].nextElementSibling;
                var time = sibling
                    ? (sibling.getElementsByTagName('time')[0] || {})
                        .innerText || '12:00'
                    : '12:00';
                var from;
                if(typeof profile !== 'undefined') {
                    from = profile.getAttribute('title');
                } else {
                    from = 'Me';
                }
                var bubbles = messages[j].getElementsByClassName(
                    'msg-s-message-listitem__message-bubble');
                for(var k = 0; k < bubbles.length; k++) {
                    result[result.length] = {
                        from: from,
                        time: times[0].innerText + ' '  + time,
                        message: bubbles[k].innerText
                    };
                }
            }
        }
        return result;
    })
    .then(r => {
        var newMessages = r.value
            .filter(e => messages
                .filter(m => m.message == e.message).length === 0);
        messages = newMessages.concat(messages);
        return newMessages.length > 0
                ? scrollLinkedInMessages(messages)
                : Promise.resolve(messages);
    })
};

var readLinkedInThread = (thread, messages) => {
    var participants;
    return client.getUrl().then(url => url.indexOf(thread) == -1
        ? client.url('https://www.linkedin.com' + thread)
            .pause(1000)
            .isExisting('a[href*="topcard"]').then(is => is 
                ? client.click('a[href*="topcard"]')
                : Promise.resolve([]))
        : (url.indexOf('topcard') == -1 
            ? client.isExisting('a[href*="topcard"]').then(is => is 
                ? client.click('a[href*="topcard"]')
                : Promise.resolve([]))
            : Promise.resolve([])))
    // get participants from topcard
    .pause(3000)
    .then(() => getThreadParticipants(thread))
    .then(p => participants = p)
    .back()
    // TODO: save to contacts
    .then(() => readLinkedInMessages(messages || []))
    .then(messages => ({
        thread: thread,
        participants: participants,
        messages: messages.map(m => {
            var newTime = chrono.parseDate(m.time);
            newTime.setHours(newTime.getHours()-(new Date()).getTimezoneOffset()/60)
            return Object.assign(m, {time: newTime})
        })
    }))
    .catch(e => console.log(e));
};
module.exports = readLinkedInThread;
readLinkedInThread;



In [1]:
var fs = require('fs');
var importer = require('../Core');
var glob = require('glob');
var path = require('path');

var PROFILE_PATH = process.env.HOME || process.env.HOMEPATH || process.env.USERPROFILE;
var project = PROFILE_PATH + '/Conversations';

var readThread = (thread) => {
    var threadId = thread.replace(/^\/|\/$/ig, '').split('/').pop();
    var file = glob.sync('**/' + threadId + '-*.json', {cwd: project})[0];
    var messages;
    try { messages = require(file) }
    catch (e) { messages = [] }
    return readLinkedInThread(thread, messages)
        .then(t => {
        var filename = project + '/' 
            + t.thread.replace(/^\/|\/$/ig, '').split('/').pop()
            + '-' + t.participants
            .map(p => p.name).join('')
            .replace(/[^a-z]/ig, '_') + '.json';
        fs.writeFileSync(filename, JSON.stringify(t, null, 4));
        return t;
    });
};

var listThreads = () => {
    return runSeleniumCell('scrape LinkedIn profile')
    .then(readLinkedInProfileInfo => readLinkedInProfileInfo())
    // show my profile info
    .then(() => runSeleniumCell('list all LinkedIn threads'))
    .then(listLinkedInThreads => listLinkedInThreads(threads))
    // list all message threads
    .then(r => fs.writeFileSync(
        project + '/threads.json',
        JSON.stringify(r, null, 4)))
};

$$.async()
var runSeleniumCell;
var threads, readLinkedInThread;

importer.interpretAll(['selenium cell'])
.then(r => {
    runSeleniumCell = r[0].runInNewContext();
    return runSeleniumCell('log in linkedin');
})
.then(loginLinkedIn => loginLinkedIn())
.then(() => {
    try { threads = JSON.parse(fs.readFileSync(project + '/threads.json')); }
    catch (e) { threads = []; }
    return runSeleniumCell('messages in LinkedIn thread');
})
//.then(() => listThreads())
// scrape each thread
.then(func => {
    readLinkedInThread = func;
    var threadCount = threads.length;
    var ids = threads.map(t => t.replace(/^\/|\/$/ig, '').split('/').pop());
    var files = glob.sync('**/@(' + ids.join('|') + ')-*.json', {cwd: project});
    var fresh = threads.filter(t => {
        var threadId = t.replace(/^\/|\/$/ig, '').split('/').pop();
        var file = files.filter(f => f.indexOf(threadId) > -1)[0];
        return !(typeof file !== 'undefined' && fs.existsSync(path.join(project, file)));
    })
    console.log((threadCount - fresh.length) + ' / ' + threadCount + ' : '
                + Math.round((threadCount - fresh.length) / threadCount * 100) + '%')
    console.log(fresh.slice(0, 50));
    if(fresh.length == threads.length && files.length !== 0) throw new Error('Wrong!');
    return importer.runAllPromises(fresh.slice(0, 50)
        .map(t => ((resolve) => readThread(t).then((r) => resolve(r)))));
})
/*
*/
.then(r => $$.sendResult(r))
.catch(e => $$.sendError(e));



Already imported
[ 'import notebook.ipynb', 'interpret all notebooks.ipynb' ]
Initializing webdriver on localhost
Already imported
[ 'import notebook.ipynb', 'interpret all notebooks.ipynb' ]
Already imported
[ 'import notebook.ipynb', 'interpret all notebooks.ipynb' ]
Initializing webdriver on localhost
Already imported
[ 'import notebook.ipynb', 'interpret all notebooks.ipynb' ]
772 / 1514 : 51%
[ '/messaging/thread/6224963803000426496/',
  '/messaging/thread/6224963747698532352/',
  '/messaging/thread/6224963686868545536/',
  '/messaging/thread/6224963520824438784/',
  '/messaging/thread/6224957701567451136/',
  '/messaging/thread/6224942697174622208/',
  '/messaging/thread/6222853475462639616/',
  '/messaging/thread/6223141365971197952/',
  '/messaging/thread/6224353463992668161/',
  '/messaging/thread/6224285838675636224/',
  '/messaging/thread/6224316729384853504/',
  '/messaging/thread/6224316694651822080/',
  '/messaging/thread/6224274110252404736/',
  '/messaging/thread/622428

[ { thread: '/messaging/thread/6224963803000426496/',
    participants: [ [Object] ],
    messages: [ [Object] ] },
  { thread: '/messaging/thread/6224963747698532352/',
    participants: [ [Object] ],
    messages: [ [Object] ] },
  { thread: '/messaging/thread/6224963686868545536/',
    participants: [],
    messages: [ [Object] ] },
  { thread: '/messaging/thread/6224963520824438784/',
    participants: [ [Object] ],
    messages: [ [Object] ] },
  { thread: '/messaging/thread/6224957701567451136/',
    participants: [ [Object] ],
    messages: [ [Object], [Object] ] },
  { thread: '/messaging/thread/6224942697174622208/',
    participants: [ [Object] ],
    messages: [ [Object] ] },
  { thread: '/messaging/thread/6222853475462639616/',
    participants: [ [Object] ],
    messages: [ [Object], [Object] ] },
  { thread: '/messaging/thread/6223141365971197952/',
    participants: [ [Object] ],
    messages: [ [Object], [Object], [Object], [Object], [Object], [Object] ] },
  { thread: 